In [1]:
!pip install pandas sodapy datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [datetime]1/3 [zope.interface]


In [3]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime,timedelta
from urllib.parse import quote
import os


# Generate query parameters for past month

In [4]:
start_date=datetime.now().replace(day=1,hour=0,minute=0,second=0,microsecond=0)
end_date = (start_date.replace(day=28) + timedelta(days=4)).replace(day=1)

start_date=start_date.strftime("%Y-%m-%dT%H:%M:%S.000")
end_date= end_date.strftime("%Y-%m-%dT%H:%M:%S.000")

where_clause = f"created_date >= '{start_date}' AND created_date <= '{end_date}'"
encoded_where = quote(where_clause)

In [5]:

BASE_URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
LIMIT = 50000  # max allowed by Socrata
OFFSET = 0
frames = []

# Calculate start date (last 30 days)
start_date = datetime(2025, 12, 1).strftime('%Y-%m-%d')
end_date = datetime(2025, 12, 31).strftime('%Y-%m-%d')
where_clause = f"created_date >= '{start_date}' AND created_date <= '{end_date}'"
encoded_where = quote(where_clause)

while True:
    url = (
        f"{BASE_URL}"
        f"?$where={encoded_where}"
        f"&$limit={LIMIT}"
        f"&$offset={OFFSET}"
    )

    chunk = pd.read_csv(url)

    if chunk.empty:
        break

    frames.append(chunk)
    OFFSET += LIMIT

    print(f"Fetched {len(chunk)} rows (offset={OFFSET})")

# Combine all pages
df_original = pd.concat(frames, ignore_index=True)

print(f"Total rows fetched: {len(df_original)}")


/tmp/ipykernel_37026/1163520192.py:20: DtypeWarning: Columns (0: facility_type, 1: due_date, 2: road_ramp) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = pd.read_csv(url)


Fetched 50000 rows (offset=50000)
Fetched 50000 rows (offset=100000)
Fetched 50000 rows (offset=150000)


/tmp/ipykernel_37026/1163520192.py:20: DtypeWarning: Columns (0: due_date, 1: taxi_company_borough) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = pd.read_csv(url)


Fetched 50000 rows (offset=200000)


/tmp/ipykernel_37026/1163520192.py:20: DtypeWarning: Columns (0: due_date, 1: taxi_company_borough, 2: bridge_highway_direction) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = pd.read_csv(url)


Fetched 50000 rows (offset=250000)


/tmp/ipykernel_37026/1163520192.py:20: DtypeWarning: Columns (0: facility_type, 1: due_date, 2: taxi_company_borough) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk = pd.read_csv(url)


Fetched 50000 rows (offset=300000)
Fetched 23343 rows (offset=350000)
Total rows fetched: 323343


In [9]:
df_original.info()

<class 'pandas.DataFrame'>
RangeIndex: 323343 entries, 0 to 323342
Data columns (total 44 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   unique_key                      323343 non-null  int64  
 1   created_date                    323343 non-null  str    
 2   closed_date                     303798 non-null  str    
 3   agency                          323343 non-null  str    
 4   agency_name                     323343 non-null  str    
 5   complaint_type                  323343 non-null  str    
 6   descriptor                      319605 non-null  str    
 7   descriptor_2                    144037 non-null  str    
 8   location_type                   292803 non-null  str    
 9   incident_zip                    321092 non-null  float64
 10  incident_address                313995 non-null  str    
 11  street_name                     313989 non-null  str    
 12  cross_street_1             

In [10]:
print(os.getcwd())

/workspaces/NYC_311_Request_DATA_ANALYSIS/Files


In [12]:
df_original.to_csv('../data/nyc_311_request_data_dec_2025.csv',index=False)